In [1]:
# Not needed if pysleep is pip installed (i.e. you probably dont need these lines)
import sys, os
file_dir = os.path.abspath('')
sys.path.insert(0, file_dir+'/../')

In [2]:
#%% Import the tools we need
from mednickdb_pysleep import sleep_features, pysleep_defaults, pysleep_utils, scorefiles, sleep_architecture
import pandas as pd
import yaml

In [3]:
#setup location of files and save locations
edf_base_path = 'X:/mednicklab_mednickdb/ExampleStudy1/raw_sleep_eeg/'
scorefile_base_path = 'X:/mednicklab_mednickdb/ExampleStudy1/sleep_scoring/'
edf_filenames = ['subject101/PinkV1 124438.edf']
scorefile_filenames = ['subjectid101_visitid1.xls']
study_settings = 'X:/mednicklab_mednickdb/ExampleStudy1/study_settings/ExampleStudy1_study_settings.yaml'
save_base_path = './'


In [4]:
#%% loop through edf's, extract spindles and then features of spindles (per stage)
edf_filepaths = [edf_base_path + edf_filename for edf_filename in edf_filenames]
scoring_filepaths = [scorefile_base_path + scorefile_filename for scorefile_filename in scorefile_filenames]
spindle_features_cont = []

study_settings = yaml.safe_load(open(study_settings,'r+'))

for edf_filepath, scorefile_filepath in zip(edf_filepaths, scoring_filepaths):   
    #%%Get stage info
    epoch_stages, epochoffset, starttime = scorefiles.extract_epochstages_from_scorefile(scorefile_filepath, 
                                                                                         study_settings['stage_map'])
    epoch_stages = scorefiles.score_wake_as_waso_wbso_wase(epoch_stages)

    #get the start and end of where we want to extract spindles from (lights off->lights on)
    lights_off_secs, lights_on_secs, \
    _, epoch_stages = sleep_architecture.lights_on_off_and_sleep_latency(epoch_stages,
                                                                         epoch_sync_offset_seconds=epochoffset)
    
    spindles = sleep_features.detect_spindles(edf_filepath, algo='Ferrarelli2007', #use a better algo!
                                              chans_to_consider=['C3','C4'], #Ferrarelli2007 sucks bus is fast
                                              start_time=lights_off_secs, 
                                              end_time=lights_on_secs)
    
    spindles = sleep_features.assign_stage_to_feature_events(spindles, epoch_stages)
    pysleep_utils.assign_quartiles(spindles, epoch_stages)
        
    spindle_features = sleep_features.sleep_feature_variables_per_stage(spindles, 
                                                                        epoch_stages, 
                                                                        av_across_channels=False)
    features_per_stage = features_per_stage.apply(lambda x: pd.to_numeric(x, errors='ignore'))
    spindle_features = spindle_features.groupby(['quartile', 'stage', 'chan', 'description']).mean()
    spindle_features_cont.append(spindle_features)
pd.concat(spindle_features_cont, axis=1).to_csv(save_base_path+'spindles')

physical_min is higher than physical_max. Check whether the polarity of your recordings is correct


NameError: name 'features_per_stage' is not defined

In [16]:
#Combine all spindles into a single dataframe and save to csv
all_spindles_info = pd.concat(spindle_features_cont, axis=0)
all_spindles_info.to_csv('PSTIM_'+drug+'_spindle_data.csv', index=False)